## Importacion de librerias

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.impute import SimpleImputer
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

## Lectura de datasets

In [3]:
orig_dir = os.getcwd()
os.chdir("..")
os.chdir('01. Dataset')
dsets_files = glob.glob('*.csv')
for fil in dsets_files:
    if fil == 'labels_bankruptcy.csv': 
        f_labels = os.path.abspath(fil)
    if fil == 'dataset.csv':
        f_data = os.path.abspath(fil)
os.chdir(orig_dir)

## Dataset y etiquetas, lectura y adaptación

In [4]:
df_col = pd.read_csv(f_labels, sep=' ',names=['codes','Names'])
# Esta conversión servirá para brevedad
names_to_codes = dict(zip(df_col['Names'],df_col['codes'])) 
codes_to_names = dict(zip(df_col['codes'],df_col['Names']))
# Se añade columna de flag ('target')
cols = ['flag'] 
cols = cols + list(df_col['Names']) 
# Se cambian los nombres a las etiquetas (estaban en chino)
df = pd.read_csv(f_data, encoding = "cp1252", names=cols, skiprows=1) 
df.head(5)

,flag,Cost_of_Interest-bearing_Debt,Cash_Reinvestment_Ratio,Current_Ratio,Acid_Test,Interest_Expenses/Total_Revenue,Total_Liability/Equity_Ratio,Liability/Total_Assets,Interest-bearing_Debt/Equity,Contingent_Liability/Equity,...,CFO_to_Assets,Cash_Flow_to_Equity,Realized_Gross_Profit_Growth_Rate,Operating_Income_Growth,Net_Income_Growth,Continuing_Operating_Income_after_Tax_Growth,Net_Income-Excluding_Disposal_Gain_or_Loss_Growth,Total_Asset_Growth,Total_Equity_Growth,Return_on_Total_Asset_Growth
0,1,-18.55,-19.50,-20.06,11.62,11.62,0.75,-35.62,-38.20,-36.37,...,-0.246930,24.739718,-6.018477,0.116169,-0.826436,2.346836,0.819707,-0.219948,0,0.426106
1,1,0.67,1.38,0.67,23.80,23.80,-1.26,0.02,-0.46,1.28,...,-0.002014,22.342296,-0.613081,0.238044,-0.004804,1.385773,226.014311,0.995576,0,0.721619
2,1,-7.17,-5.81,-7.63,11.61,11.49,-8.93,-70.59,-70.60,-61.66,...,-0.066410,107.094021,0.710991,0.116115,-0.222129,2.344816,0.776262,-0.288225,0,0.426473
3,1,-12.55,-14.57,-10.35,-13.24,-13.24,-22.57,-29.85,-26.20,-7.28,...,-0.176035,8.776406,-5.670492,-0.132425,-0.363423,1.064489,0.910554,-0.098232,0,0.939418
4,1,0.82,1.42,1.71,7.91,7.91,1.09,-1.00,-0.23,-2.09,...,-0.002892,10.448692,-1.529697,0.079096,-0.004567,0.579505,-0.929778,2.075525,0,1.725610


## Preprocesamiento

In [5]:
# Cambiamos los nombres de columnas por facilidad : 
# x1 a x95 atributos y Y como etiqueta de quiebra o no quiebra (flag)
df_bankruptcy = df.rename(columns=names_to_codes,inplace=False)
df_bankruptcy.head(5)

,flag,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95
0,1,-18.55,-19.50,-20.06,11.62,11.62,0.75,-35.62,-38.20,-36.37,...,-0.246930,24.739718,-6.018477,0.116169,-0.826436,2.346836,0.819707,-0.219948,0,0.426106
1,1,0.67,1.38,0.67,23.80,23.80,-1.26,0.02,-0.46,1.28,...,-0.002014,22.342296,-0.613081,0.238044,-0.004804,1.385773,226.014311,0.995576,0,0.721619
2,1,-7.17,-5.81,-7.63,11.61,11.49,-8.93,-70.59,-70.60,-61.66,...,-0.066410,107.094021,0.710991,0.116115,-0.222129,2.344816,0.776262,-0.288225,0,0.426473
3,1,-12.55,-14.57,-10.35,-13.24,-13.24,-22.57,-29.85,-26.20,-7.28,...,-0.176035,8.776406,-5.670492,-0.132425,-0.363423,1.064489,0.910554,-0.098232,0,0.939418
4,1,0.82,1.42,1.71,7.91,7.91,1.09,-1.00,-0.23,-2.09,...,-0.002892,10.448692,-1.529697,0.079096,-0.004567,0.579505,-0.929778,2.075525,0,1.725610


In [6]:
# Verificamos los tipos sobretodo interesa que flag sea entero
print(df_bankruptcy.dtypes) 

flag      int64
X1      float64
X2      float64
X3      float64
X4      float64
         ...   
X91     float64
X92     float64
X93     float64
X94       int64
X95     float64
Length: 96, dtype: object


In [15]:
# Quitamos X94 ya que no se usará, dado que no es relevante
df_bankruptcy = df_bankruptcy.drop(['X94'], axis=1) 

In [7]:
# Convertimos todas las columnas a float por precaucion
def convert_datatype(df):
    index = 1
    while (index <= df_bankruptcy.shape[1]-1):
        colname = df.columns[index]
        col = getattr(df, colname)
        df[colname] = col.astype(float)
        index += 1


convert_datatype(df_bankruptcy)

In [8]:
# Convertimos las etiquetas a int (tambien por precaucion)
def labels_to_binary(df):
    col = getattr(df, 'flag')
    df['flag'] = col.astype(int)

labels_to_binary(df_bankruptcy)

In [9]:
# Verificamos la existencia de nulos en el dataset
df_bankruptcy.isnull().sum()/df_bankruptcy.shape[0]*100 

flag    0.0
X1      0.0
X2      0.0
X3      0.0
X4      0.0
       ... 
X91     0.0
X92     0.0
X93     0.0
X94     0.0
X95     0.0
Length: 96, dtype: float64

In [10]:
# Verificamos que no existen datos nulos
df_bankruptcy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flag    6819 non-null   int32  
 1   X1      6819 non-null   float64
 2   X2      6819 non-null   float64
 3   X3      6819 non-null   float64
 4   X4      6819 non-null   float64
 5   X5      6819 non-null   float64
 6   X6      6819 non-null   float64
 7   X7      6819 non-null   float64
 8   X8      6819 non-null   float64
 9   X9      6819 non-null   float64
 10  X10     6819 non-null   float64
 11  X11     6819 non-null   float64
 12  X12     6819 non-null   float64
 13  X13     6819 non-null   float64
 14  X14     6819 non-null   float64
 15  X15     6819 non-null   float64
 16  X16     6819 non-null   float64
 17  X17     6819 non-null   float64
 18  X18     6819 non-null   float64
 19  X19     6819 non-null   float64
 20  X20     6819 non-null   float64
 21  X21     6819 non-null   float64
 22  

In [26]:
# Revisamos si hay duplicados
df_bankruptcy.duplicated().sum()

0

In [11]:
# Función para eliminar los valores Nan
def drop_nans(df_bankruptcy, verbose=False):
    clean_dataframes = df_bankruptcy.dropna(axis=0, how='any')
    return clean_dataframes

nan_dropped_df = drop_nans(df_bankruptcy, verbose=True)

In [19]:
# Función para imputar nulos, considerando la mediana dada la distribuición de los datos
def imputation(df, strategy='median'):
    imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)
    mean_imputed_df = pd.DataFrame(imputer.fit_transform(df))
    mean_imputed_df.columns = df.columns
    return mean_imputed_df

median_imputed_df = imputation(df_bankruptcy)

In [23]:
# Se podrian emplear df combinados como un dict ordenado, a codificar en el futuro
imputed_dict = OrderedDict()
imputed_dict['Median'] = median_imputed_df
median_imputed_df.head(5)

,flag,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95
0,1.0,-18.55,-19.50,-20.06,11.62,11.62,0.75,-35.62,-38.20,-36.37,...,-0.246930,24.739718,-6.018477,0.116169,-0.826436,2.346836,0.819707,-0.219948,0.0,0.426106
1,1.0,0.67,1.38,0.67,23.80,23.80,-1.26,0.02,-0.46,1.28,...,-0.002014,22.342296,-0.613081,0.238044,-0.004804,1.385773,226.014311,0.995576,0.0,0.721619
2,1.0,-7.17,-5.81,-7.63,11.61,11.49,-8.93,-70.59,-70.60,-61.66,...,-0.066410,107.094021,0.710991,0.116115,-0.222129,2.344816,0.776262,-0.288225,0.0,0.426473
3,1.0,-12.55,-14.57,-10.35,-13.24,-13.24,-22.57,-29.85,-26.20,-7.28,...,-0.176035,8.776406,-5.670492,-0.132425,-0.363423,1.064489,0.910554,-0.098232,0.0,0.939418
4,1.0,0.82,1.42,1.71,7.91,7.91,1.09,-1.00,-0.23,-2.09,...,-0.002892,10.448692,-1.529697,0.079096,-0.004567,0.579505,-0.929778,2.075525,0.0,1.725610


In [36]:
# Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html
# Estandarizamos el dataset
from sklearn.preprocessing import scale
X = scale(median_imputed_df.drop('flag', axis=1))

dataset_normalizado = pd.DataFrame(X, index=median_imputed_df.index, columns=median_imputed_df.columns[1:]).head()
dataset_normalizado['flag'] = median_imputed_df['flag']
dataset_normalizado.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X87,X88,X89,X90,X91,X92,X93,X94,X95,flag
0,-2.217909,-2.045798,-2.400361,-0.383334,-0.382638,0.016456,-0.023516,-0.020165,-0.087479,-0.031281,...,0.017106,-0.084274,-0.383465,-0.861611,0.680171,-0.060031,-0.098978,0.0,-0.622067,1.0
1,-0.673828,-0.311068,-0.598450,0.135068,0.136307,0.014671,0.014799,0.015966,-0.005957,0.009823,...,-0.009316,-0.021388,0.135253,-0.029804,0.240685,15.129816,0.364559,0.0,-0.535573,1.0
2,-1.303672,-0.908420,-1.319910,-0.383759,-0.388177,0.007858,-0.061111,-0.051183,-0.142238,-0.086566,...,0.924712,-0.005984,-0.383698,-0.249820,0.679247,-0.062961,-0.125015,0.0,-0.621960,1.0
3,-1.735886,-1.636209,-1.556340,-1.441418,-1.441832,-0.004257,-0.017313,-0.008676,-0.024491,-0.011077,...,-0.158822,-0.080226,-1.441526,-0.392864,0.093765,-0.053903,-0.052562,0.0,-0.471826,1.0
4,-0.661778,-0.307745,-0.508050,-0.541238,-0.540708,0.016758,0.013703,0.016186,-0.013254,0.013300,...,-0.140392,-0.032052,-0.541255,-0.029564,-0.128014,-0.178037,0.776395,0.0,-0.241715,1.0


In [16]:
# Guardamos la estructura de datasets (para futuro)
pickle.dump(imputed_dict,open(os.path.join('results','imputed_data.pkl'),'wb'))

In [37]:
# Guardamos el dataframe imputado
median_imputed_df.to_pickle(os.path.join('results','dataset_normalizado.pkl'))